In [76]:
!pip install google_trans_new

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import google_trans_new

In [4]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanjiatong/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hanjiatong/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hanjiatong/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/hanjiatong/nltk_data...


True

In [6]:
df = pd.read_csv('../../Datasets/cleaned_customer.csv')

In [7]:
df.head()

,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,host_since,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating
0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,...,10.0,8.0,8.0,f,2,0.02,0.995731,0.617762,4.295315,4.595264
1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
2,15222393,50646,2014-06-05,Welli,Stayed over at Sujatha's house for 3 good nigh...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
3,5543172,50646,2014-07-02,Cyril,It's been a lovely stay at Sujatha's. The room...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
4,817532,50646,2014-07-08,Jake,"We had a great experience. A nice place, an am...",Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423


In [9]:
df.comments.isna().value_counts()

False    155078
True        172
Name: comments, dtype: int64

In [10]:
df.comments.fillna('',inplace=True)

#### Detect foreign languages and translate them into English

In [87]:
from google_trans_new import google_translator 
translator = google_translator() 

def detectandtranslate(text):
    if translator.detect(text)[1] != 'english':
        return translator.translate(text,lang_tgt='en')
    else:
        return text

### 1. Tokenize the text and extract most frequent words

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

def tokenizer(text):
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    return(text)

vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range=(1,2),
                             max_df = 0.6,
                             tokenizer=tokenizer,
                             max_features=500)
X = vectorizer.fit_transform(df.comments)

In [25]:
print(vectorizer.get_feature_names())

['\n\nthe', '\nthe', 'able', 'absolutely', 'ac', 'access', 'accessible', 'accommodating', 'accommodation', 'accurate', 'actually', 'adults', 'advertised', 'affordable', 'air', 'airbnb', 'aircon', 'airport', 'alex', 'allowed', 'alvin', 'amazing', 'amenities', 'apartment', 'appartment', 'appreciated', 'area', 'areas', 'arrival', 'arrived', 'ask', 'asked', 'attractions', 'automated', 'available', 'away', 'awesome', 'bad', 'balcony', 'bars', 'basic', 'bathroom', 'bathrooms', 'bay', 'beautiful', 'bed', 'bedroom', 'bedrooms', 'beds', 'best', 'better', 'bien', 'big', 'bigger', 'bit', 'book', 'booked', 'booking', 'breakfast', 'bring', 'budget', 'bugis', 'building', 'bus', 'buses', 'business', 'busy', 'cafes', 'came', "can't", 'canceled', 'care', 'center', 'central', 'centrally', 'centre', 'chance', 'cheap', 'check', 'check-in', 'checked', 'checking', 'children', 'chinatown', 'choice', 'choose', 'city', 'clean', 'cleaned', 'cleaning', 'cleanliness', 'clear', 'close', 'clothes', 'coffee', 'cold'

In [26]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### 2. Using Vader Model
Reasons for choosing this model:<br>
1. From the NLTK package, Vadar model is trained with web comments, reviews, making itself suitable for this senario.
2. It accounts for many acronyms, slangs, Capitalized Words and even emojis that are recently prevalent on the Internet.
3. Rule-based model makes it easily adjustable.

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def get_sentiment(sentence,sid):
    score = sid.polarity_scores(sentence)
    compound = score.get('compound')
    return compound

In [15]:
sid = SentimentIntensityAnalyzer()
newWords = {'amenities': 0.5, 'big': 0.5, 'bus': 0.5,'near':0.5,'nearby':0.5,'quiet':0.5,'quick':0.5,'spacious':0.5,'mrt':0.5,'station':0.5,'walking':0.5,'walk':0.5,'restaurant':0.5,'responsive':0.5,'shopping':0.5,'short':0.5,
    'pool':0.5,'kitchen':0.5,'food':0.5,'family':0.5,'distance':0.5,'close':0.5,'communication':0.5}
sid.lexicon.update(newWords)

In [16]:
get_sentiment('good amenities',sid)

0.5267

In [17]:
sid_score = df.comments.apply(lambda x : get_sentiment(x,sid))

In [18]:
sid_score.describe()

count    155250.000000
mean          0.645322
std           0.402002
min          -0.998300
25%           0.440400
50%           0.839500
75%           0.945500
max           0.999800
Name: comments, dtype: float64

In [40]:
df.comments[sid_score >= 0.99][31]

"I'm a French student who was looking for a place for some days in waiting to find a room nearby my exchange university and my stay in Fran's room has been my first Airbnb experience. To resume : it was PERFECT ! I met fantastic people (Fran and her family), helpful, kind, respectful, etc.. They are easygoing, you can speak with them about any topic (travels, music, religion..). They hosted me longer than it was planned and I thank them for it. Now I consider them as a second family in Singapore. I thank them for all advices they gave me about Singapore too. \nAbout the room and the flat, it was perfect too. A very nice place, clean, with all you need (MRT at 10 min walking distance,Foodcourt, Supermarket, etc...). \nIf  you look for a place to stay (outside the city), it is the place to go !\nTo finish : Thank you for all Fran and see you soon :)"

In [52]:
pd.set_option('max_colwidth',500)
df.comments[sid_score <= -0.99] # mostly in German

765       Ich war 3 Wochen lang bei Familie Woon untergekommen und hatte eine sehr schöne Zeit. Am Abend meiner Ankunft wurde ich vom mittleren Sohn, Gabriel, empfangen und in die Unterkunft eingeführt. Die Wohnung liegt im 7. Stock eines Appartmentblocks, ist voll klimatisiert und es kommen ca. 11 Personen dort unter (5 Familienmitglieder und Gäste) wobei es einem nicht vollgestopft vorkommt. Die Familie wie auch die Gäste waren alle sehr sehr nett, und gemeinsam Essen oder Feiern gehen war stets ein...
2977      Das Top-Erlebnis über Weihnachten in Singapur. Charlyn und Cecilia sind grossartige Hosts - und ihr Angebot nur zu empfehlen! Die Wohnung ist etwas ausserhalb des Zentrums der Metropole mit seinen unzähligen Einkaufscentern - auf halbem Weg vom Flughafen Changi - im Osten der Insel. Eine schöne, sichere und leicht hippe Wohngegend, wofür Charlyns Mutter Cecilia die besten Restauranttipps immer gleich griffbereit auf ihrem Handy hat. Charlyn war ziemlich busy während meines Au

In [95]:
df['senti_score'] = sid_score

In [92]:
df.senti_score[sid_score <= -0.99] = df.comments[sid_score <= -0.99].apply(lambda x:get_sentiment(detectandtranslate(x),sid))

google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [ ]:
df.to_csv('../../Datasets/cleaned_customer.csv')